In [ ]:
!pip install -U langchain-dartmouth > /dev/null
!pip install faiss-cpu > /dev/null

from langchain_dartmouth.llms import ChatDartmouth, DartmouthLLM
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_dartmouth.embeddings import DartmouthEmbeddings

from langchain.docstore.document import Document
from langchain_dartmouth.retrievers.document_compressors import DartmouthReranker

from langchain_community.vectorstores import FAISS
import faiss
import json

from sentence_transformers import SentenceTransformer

import os
import numpy as np

In [ ]:
os.environ["DARTMOUTH_API_KEY"] = "KEY_GOES_HERE"

In [ ]:
def format_prompt(prompt):
    """
    This function will format a prompt into what is needed for LangChain to produce ChatML.
    Args:
       prompt: the text to be embedded as human prompt.
    """
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=prompt),
    ]
    return messages

In [ ]:
max_new_tokens = 1024
top_p = 0.95 # If set to < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
temperature = .95 #  Strictly positive float value used to modulate the logits distribution. A value smaller than 1 decreases randomness (and vice versa), with 0 being equivalent to shifting all probability mass to the most likely token
repetition_penalty = None

system_prompt = "Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."
kwargs = dict()

llm_chat = ChatDartmouth(model_name="llama-3-1-8b-instruct",
                    temperature = temperature,
                    top_p = top_p,
                    max_tokens = max_new_tokens,
                    model_kwargs=kwargs)

In [ ]:
# download LayupList data
!wget -O /content/old_reviews.json https://raw.githubusercontent.com/jeddobson/ENGL64.05-22F/refs/heads/main/data/LayupList/old_reviews.json

In [ ]:
# open older format reviews and extract comments
reviews = json.loads(open("old_reviews.json").read())
reviews_text= [r["comments"]["oldReview"] for r in reviews if 'comments' in r]
# how many did we find?
print("found {0} reviews".format(len(reviews_text)))

In [ ]:
# load a smaller embedding model that will quickly embed all our documents
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Create document embeddings with embedding model
doc_embeddings = embedding_model.encode(reviews_text)

In [ ]:
# display number of documents and embedding width
doc_embeddings.shape

In [ ]:
# Build FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

# this will retrieve five closest neighbors using document similarity
def retrieve_documents(query, k=5):
    query_embedding = embedding_model.encode([query])[0]
    distances, indices = index.search(np.array([query_embedding]), k)
    return [reviews_text[i] for i in indices[0]]

In [ ]:
query = "I am interested in the very best courses in Chemistry Department. What are the really good courses taught by awesome professors?"

# Retrieve relevant documents for our query
retrieved_docs = retrieve_documents(query,k=10)

# Join query with context.
context = query + "\n" + "\n".join(retrieved_docs)

In [ ]:
# Generate!
output = llm_chat.invoke(format_prompt(context))
print(output.content)

In [ ]:
# Display context sent to LLM
context